In [ ]:
import geopandas as gpd
from pathlib import Path

# directory containing your raw GeoJSON files
indir = Path("/datawaha/esom/DatePalmCounting/Geoportal/Center_pivot/year_base_consistent")

# output directory
outdir = indir  # or Path(".../simpl/")

years = [1990,1995,2000,2005,2010,2015,2016,2017,2018,2019,2020,2021,2022,2023]

# tolerance (meters) after reprojection
TOL = 30  # try 10–30 for best results

for y in years:
    infile = indir / f"CPF_fields_{y}.geojson"     # or CPF_fileds_{y}.geojson
    if not infile.exists():
        infile = indir / f"CPF_fileds_{y}.geojson"
    if not infile.exists():
        print(f"❌ Missing: {y}")
        continue

    print(f"→ Simplifying {infile.name}")

    # read
    gdf = gpd.read_file(infile)

    # project to WebMercator for metric simplification
    gdf_m = gdf.to_crs(3857)

    # simplify geometry
    gdf_m["geometry"] = gdf_m.geometry.simplify(
        TOL, preserve_topology=True
    )

    # back to WGS84
    gdf_s = gdf_m.to_crs(4326)

    outfile = outdir / f"CPF_fields_{y}_simpl.geojson"
    gdf_s.to_file(outfile, driver="GeoJSON")

    print(f"   ✅ Saved: {outfile}")
